In [96]:
import glob

exclude = ['./CLAUDE.md','./progress-tracker.md','./project-description.md']
list_of_files = glob.glob('meridian-islands/**/*.md') 
filtered_list = []
for a in list_of_files:
    if a not in exclude:
        filtered_list.append(a)

filtered_list = sorted(filtered_list)

current_list = filtered_list[0]
print(current_list)

markdown_string = " "

with open(current_list, "r", encoding="utf-8") as f:
    first_text = f.read() 

with open("first.txt", "w", encoding="utf-8") as text_file:
    text_file.write(first_text)

meridian-islands/01-foundation/world-bible.md


In [101]:
for start in range(0,len(first_text),200):
    end = min(start+200, len(first_text))
    chunk = first_text[start:end]

chunks = []
i=0
for start in (range(0, len(first_text), 200)):
    end = min(start+200, len(first_text))
    chunk = first_text[start:end]
    chunks.append({
        "text": chunk,
        "chunk_id": i+1,
        "start_pos": start,
        "end_pos": end-1,
        "char_count": len(chunk),
        "origin_file": current_list
    })
    i += 1

